In [5]:
#get proper modules
import pandas as pd
import numpy as np
import sklearn as sk



#load data (all feature should be combined to one dframe here)
data = pd.read_csv("datasets/amalgamated_data.csv")

for index, row in data.iterrows():
    state = (row['state'])
    state = state[1:len(state)-1] 
    data.set_value(index, 'state', state)

#show head to ensure proper import
data.head()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  from ipykernel import kernelapp as app


,Unnamed: 0,latitude,longitude,barometric pressure noons:sea level pressure,barometric pressure noons:pressure,month,year,state,humidities:relative humidity (%),humidities:dewpoint,air_temp
0,1009166,39.724047,-77.523373,983.1,955.0,11,2017,Pennsylvania,55.0,-11.5,7.9
1,725218,39.724047,-77.523373,980.1,952.1,11,2017,Pennsylvania,59.0,-1.2,-1.3
2,8176112,34.151827,-118.130506,1014.9,1012.2,2,2016,California,68.0,11.2,13.1
3,4143373,43.700000,-72.100000,1000.1,997.2,10,2014,New Hampshire,83.0,-2.9,4.1
4,7782185,41.810378,-71.449503,997.5,994.6,12,2017,Rhode Island,55.0,-20.4,9.1


In [3]:
#get proper modules
import pandas as pd
import numpy as np
import sklearn as sk

health = pd.read_csv("datasets/health_data.csv")

#data.to_csv("datasets/amalgamated_data.csv")
data = pd.read_csv("datasets/amalgamated_data.csv")

#merge data and health
data = data.merge(health, left_on=['state','month','year'], right_on=['State','month','Year'])


In [4]:
data.head()

,Unnamed: 0,Unnamed: 0.1,latitude,longitude,barometric pressure noons:sea level pressure,barometric pressure noons:pressure,month,year,state,humidities:relative humidity (%),humidities:dewpoint,air_temp,index,State,Year,Infected
0,0,1009166,39.724047,-77.523373,983.1,955.0,11,2017,Pennsylvania,55.0,-11.5,7.9,19386,Pennsylvania,2017,60038
1,0,1009166,39.724047,-77.523373,983.1,955.0,11,2017,Pennsylvania,55.0,-11.5,7.9,19439,Pennsylvania,2017,62684
2,0,1009166,39.724047,-77.523373,983.1,955.0,11,2017,Pennsylvania,55.0,-11.5,7.9,19492,Pennsylvania,2017,63190
3,0,1009166,39.724047,-77.523373,983.1,955.0,11,2017,Pennsylvania,55.0,-11.5,7.9,19545,Pennsylvania,2017,55326
4,1,725218,39.724047,-77.523373,980.1,952.1,11,2017,Pennsylvania,59.0,-1.2,-1.3,19386,Pennsylvania,2017,60038


In [5]:
data.drop(data.columns[[0,1,2,11,13]], 1, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000611 entries, 0 to 2000610
Data columns (total 11 columns):
longitude                                       float64
barometric pressure noons:sea level pressure    float64
barometric pressure noons:pressure              float64
month                                           int64
year                                            int64
state                                           object
humidities:relative humidity (%)                float64
humidities:dewpoint                             float64
index                                           int64
Year                                            int64
Infected                                        int64
dtypes: float64(5), int64(5), object(1)
memory usage: 183.2+ MB


In [8]:
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline 
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin

def split_train_set(data, test_ratio):
    shuf_index = np.random.permutation(len(data))
    test_size = int(len(data) * test_ratio)
    test_indexes = shuf_index[:test_size]
    train_indexes = shuf_index[test_size:]
    return data.iloc[train_indexes], data.iloc[test_indexes]

#handle dataframe and numpy array transformations
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def _init_(self, attribute_names):
        self.atrribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

#drop nulls (failsafe)
data.dropna()

#spilt training and test set (USE TRAIN FOR ALL REGRESSORS)
train_data, test_data = split_train_set(data, .2)

#seperate labels and attributes 
train_data_labels = train_data['Infected']
train_data.drop(train_data.columns[[9]], 1, inplace=True)
train_data.drop(['state'], 1, inplace = True)

test_data_labels = test_data['Infected']
test_data.drop(test_data.columns[[9]], 1, inplace=True)
test_data.drop(['state'], 1, inplace = True)

train_data.info()

#add additional transformation tasks to the below sklearn pipeline
#standardize data (use devide by variance standardization to minimize outlier impact)
stpipeline = Pipeline([
    ('std_scaler', StandardScaler()),
])
train_data_prepped = stpipeline.fit_transform(train_data)
test_data_prepped = stpipeline.fit_transform(test_data)



<class 'pandas.core.frame.DataFrame'>
Int64Index: 1600489 entries, 1180555 to 238898
Data columns (total 9 columns):
longitude                                       1600489 non-null float64
barometric pressure noons:sea level pressure    1600489 non-null float64
barometric pressure noons:pressure              1600489 non-null float64
month                                           1600489 non-null int64
year                                            1600489 non-null int64
humidities:relative humidity (%)                1600489 non-null float64
humidities:dewpoint                             1600489 non-null float64
index                                           1600489 non-null int64
Infected                                        1600489 non-null int64
dtypes: float64(5), int64(4)
memory usage: 122.1 MB


/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

def display_scores(scores):
    print("Score: ")
    print(scores)
    print("\nMean: ")
    print(scores.mean())
    print("\nStd. Dev.: ") 
    print(scores.std())
    
lin_reg = LinearRegression() 
lin_reg.fit(train_data_prepped, train_data_labels)

lin_scores = cross_val_score(lin_reg, train_data_prepped, train_data_labels, 
    scoring = "neg_mean_squared_error", cv=10) 
lin_rmse_scores = np.sqrt(-lin_scores)

display_scores(lin_rmse_scores)


Score: 
[1.82010948e-08 1.64722948e-08 3.22974286e-08 2.38074058e-08
 2.36827807e-08 3.53687600e-08 2.33630346e-08 2.42335950e-08
 2.51181019e-08 3.83258043e-08]

Mean: 
2.608703005035595e-08

Std. Dev.: 
6.722421882309161e-09


In [50]:
forest_reg = RandomForestRegressor() 
forest_reg.fit(train_data_prepped, train_data_labels)

forest_scores = cross_val_score(forest_reg, train_data_prepped, train_data_labels, 
    scoring = "neg_mean_squared_error", cv=10) 
forest_rmse_scores = np.sqrt(-scores)

display_scores(forest_rmse_scores)

/usr/local/lib/python3.7/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


KeyboardInterrupt: 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400122 entries, 1023644 to 703494
Data columns (total 6 columns):
barometric pressure noons:sea level pressure    400122 non-null float64
barometric pressure noons:pressure              400122 non-null float64
month                                           400122 non-null int64
year                                            400122 non-null int64
humidities:relative humidity (%)                400122 non-null float64
humidities:dewpoint                             400122 non-null float64
dtypes: float64(4), int64(2)
memory usage: 21.4 MB


/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


1023644    65665
9881       62684
323844      4285
615287      4930
932311     59874
Name: Infected, dtype: int64

In [11]:
from sklearn.metrics import mean_squared_error
lin_prediction = lin_reg.predict(test_data_prepped)
mse = mean_squared_error(test_data_labels, lin_prediction)
rmse = np.sqrt(mse)

print(rmse)

NameError: name 'mean_squared_error' is not defined